# Getting Query from PubMed

In [2]:
from pathlib import Path
import requests
import time

# 1. Create a list of all the URLs you want to download
URLS = [
    "https://pubmed.ncbi.nlm.nih.gov/27242579/",
    "https://pubmed.ncbi.nlm.nih.gov/32457675/",
    "https://pubmed.ncbi.nlm.nih.gov/32528365/"
]

# Set browser-like headers to avoid being blocked by the site
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    ),
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://pubmed.ncbi.nlm.nih.gov/",
}

# Create a directory to store the output files
output_dir = Path("html_outputs")
output_dir.mkdir(exist_ok=True)


# Use a single session for all requests for efficiency
with requests.Session() as s:
    s.headers.update(headers)
    
    # 2. Loop through each URL in the list
    for i, url in enumerate(URLS):
        print(f"Downloading URL {i+1}/{len(URLS)}: {url[:50]}...")
        
        try:
            # 3. Create a unique filename for each URL
            outfile = output_dir / f"output_{i+1}.html"

            resp = s.get(url, timeout=30)
            resp.raise_for_status()  # raise an error for non-200 responses

            # Use server-provided encoding when available; default to utf-8
            if not resp.encoding:
                resp.encoding = "utf-8"
                
            outfile.write_text(resp.text, encoding=resp.encoding)

            print(f"-> Saved {outfile.resolve()} ({outfile.stat().st_size:,} bytes)\n")

        except requests.exceptions.RequestException as e:
            print(f"!! Failed to download URL {i+1}. Error: {e}\n")
        
        # Optional: Add a small delay to be respectful to the server
        time.sleep(1)

print("All downloads complete.")

-> Saved C:\Users\User\Desktop\Psychoinformatics Neuroinformatics\ETL_HW\html_outputs\output_1.html (134,094 bytes)

-> Saved C:\Users\User\Desktop\Psychoinformatics Neuroinformatics\ETL_HW\html_outputs\output_2.html (138,335 bytes)

-> Saved C:\Users\User\Desktop\Psychoinformatics Neuroinformatics\ETL_HW\html_outputs\output_3.html (128,509 bytes)

All downloads complete.


# Extracting PMID from the HTMLs

In [3]:
from bs4 import BeautifulSoup
from pathlib import Path

# 1. Define the directory containing your HTML files
INPUT_DIR = Path("html_outputs")

# 2. Create a list to store the results as dictionaries
extraction_results = []

# 3. Find all .html files in the directory and loop through them
html_files = list(INPUT_DIR.glob("*.html"))
print(f"Found {len(html_files)} HTML files to process in '{INPUT_DIR}'...\n")

for html_file in html_files:
    print(f"Processing: {html_file.name}")
    
    html_text = html_file.read_text(encoding='utf-8', errors='ignore')
    soup = BeautifulSoup(html_text, 'html.parser')
    
    # Find the specific meta tag
    meta = soup.find('meta', attrs={'name': 'log_displayeduids'})
    
    if meta:
        pmids_str = meta.get('content', '')
        if pmids_str:
            # Split the string by comma to get individual IDs
            pmids = pmids_str.split(',')
            
            # For each PMID found, add it to our results with its source file
            for pmid in pmids:
                cleaned_pmid = pmid.strip()
                if cleaned_pmid: # Ensure it's not an empty string
                    extraction_results.append({
                        'source_file': html_file.name,
                        'pmid': cleaned_pmid
                    })
            
            print(f"  -> Found {len(pmids)} PMIDs.\n")
        else:
            print("  -> Meta tag found, but it has no content.\n")
    else:
        print(f"  -> WARNING: Could not find the 'log_displayeduids' meta tag in this file.\n")

## **Extraction Results**

# 4. Print the final results directly to the console
print(f"{'Source File':<25} | {'PMID'}")
print(f"{'-'*25}-|--------")

if not extraction_results:
    print("No PMIDs were found.")
else:
    for item in extraction_results:
        print(f"{item['source_file']:<25} | {item['pmid']}")

print("\n" + "="*40)
print("Processing complete!")
print(f"Total PMIDs extracted: {len(extraction_results)}")
extraction_results[:3]
pmids_only = [item['pmid'] for item in extraction_results]
print(pmids_only)


Found 6 HTML files to process in 'html_outputs'...

Processing: After the Honeymoon.html
  -> Found 1 PMIDs.

Processing: Attractive Alternative Partners.html
  -> Found 1 PMIDs.

Processing: Lucky Guy in Love.html
  -> Found 1 PMIDs.

Processing: output_1.html
  -> Found 1 PMIDs.

Processing: output_2.html
  -> Found 1 PMIDs.

Processing: output_3.html
  -> Found 1 PMIDs.

Source File               | PMID
--------------------------|--------
After the Honeymoon.html  | 32457675
Attractive Alternative Partners.html | 32528365
Lucky Guy in Love.html    | 27242579
output_1.html             | 27242579
output_2.html             | 32457675
output_3.html             | 32528365

Processing complete!
Total PMIDs extracted: 6
['32457675', '32528365', '27242579', '27242579', '32457675', '32528365']


# Extracting PMC ID from HTMLs

In [4]:
from bs4 import BeautifulSoup
from pathlib import Path

# 1. Define the directory containing your HTML files
INPUT_DIR = Path("html_outputs")

# 2. Create a list to store the results
pmc_results = []

# 3. Find all .html files in the directory and loop through them
html_files = list(INPUT_DIR.glob("*.html"))
print(f"Found {len(html_files)} HTML files to process in '{INPUT_DIR}'...\n")

for html_file in html_files:
    print(f"Processing: {html_file.name}")
    
    html_text = html_file.read_text(encoding='utf-8', errors='ignore')
    soup = BeautifulSoup(html_text, 'html.parser')
    
    # Find the meta tag with name="keywords"
    keywords_meta_tag = soup.find('meta', attrs={'name': 'keywords'})
    
    pmc_id_found = False
    if keywords_meta_tag:
        # Get the content string, which contains pmid, pmcid, doi, etc.
        content_str = keywords_meta_tag.get('content', '')
        
        # Split the string by commas to get individual parts
        content_parts = content_str.split(',')
        
        # Loop through the parts to find the one starting with "PMC"
        for part in content_parts:
            # .strip() removes any leading/trailing whitespace
            cleaned_part = part.strip()
            if cleaned_part.startswith('PMC'):
                pmc_results.append({
                    'source_file': html_file.name,
                    'pmc_id': cleaned_part
                })
                print(f"  -> Found PMC ID: {cleaned_part}\n")
                pmc_id_found = True
                break # Stop searching once the PMC ID is found
    
    # If the loop finishes and no PMC ID was found
    if not pmc_id_found:
        print(f"  -> No PMC ID found in this file.\n")

## **Extraction Results**

# 4. Print the final results directly to the console
print(f"{'Source File':<25} | {'PMC ID'}")
print(f"{'-'*25}-|-----------")

if not pmc_results:
    print("No PMC IDs were found in any of the files.")
else:
    for item in pmc_results:
        print(f"{item['source_file']:<25} | {item['pmc_id']}")

print("\n" + "="*40)
print("Processing complete!")
print(f"Total files with PMC IDs: {len(pmc_results)}")

Found 6 HTML files to process in 'html_outputs'...

Processing: After the Honeymoon.html
  -> Found PMC ID: PMC7223160

Processing: Attractive Alternative Partners.html
  -> Found PMC ID: PMC7264388

Processing: Lucky Guy in Love.html
  -> Found PMC ID: PMC4863427

Processing: output_1.html
  -> Found PMC ID: PMC4863427

Processing: output_2.html
  -> Found PMC ID: PMC7223160

Processing: output_3.html
  -> Found PMC ID: PMC7264388

Source File               | PMC ID
--------------------------|-----------
After the Honeymoon.html  | PMC7223160
Attractive Alternative Partners.html | PMC7264388
Lucky Guy in Love.html    | PMC4863427
output_1.html             | PMC4863427
output_2.html             | PMC7223160
output_3.html             | PMC7264388

Processing complete!
Total files with PMC IDs: 6


# Extracting the keywords from HTMLs

In [6]:
from bs4 import BeautifulSoup, NavigableString
from pathlib import Path
import re

# 1. Define the directory containing your HTML files
INPUT_DIR = Path("html_outputs")

# 2. Create a list to store the results
keyword_results = []

# 3. Find all .html files and loop through them
html_files = list(INPUT_DIR.glob("*.html"))
print(f"Found {len(html_files)} HTML files to process in '{INPUT_DIR}'...\n")

for html_file in html_files:
    print(f"Processing: {html_file.name}")
    
    html_text = html_file.read_text(encoding='utf-8', errors='ignore')
    soup = BeautifulSoup(html_text, 'html.parser')
    
    # Find the <strong> tag that contains the text "Keywords:"
    # We use a regex with re.IGNORECASE to match "Keywords:" or "keywords:"
    keyword_tag = soup.find('strong', string=re.compile(r'Keywords:', re.IGNORECASE))
    
    keywords_found = False
    if keyword_tag:
        # The keywords are in the text node immediately following the <strong> tag
        next_element = keyword_tag.next_sibling
        
        # Check if the next element is actually text (a NavigableString)
        if next_element and isinstance(next_element, NavigableString):
            # .strip() removes whitespace, .rstrip('.') removes the final period
            keyword_str = next_element.strip().rstrip('.')
            
            # Split the string by semicolon and clean up each keyword
            keywords = [kw.strip() for kw in keyword_str.split(';')]
            
            keyword_results.append({
                'source_file': html_file.name,
                'keywords': keywords
            })
            print(f"  -> Found {len(keywords)} keywords.\n")
            keywords_found = True

    if not keywords_found:
        print(f"  -> No keywords section found in this file.\n")

## **Extraction Results**

# 4. Print the final results
print(f"{'Source File':<25} | {'Keywords'}")
print(f"{'-'*25}-|-----------")

if not keyword_results:
    print("No keywords were found in any of the files.")
else:
    for item in keyword_results:
        # ', '.join() converts the list of keywords into a nice string for printing
        keywords_str = ', '.join(item['keywords'])
        print(f"{item['source_file']:<25} | {keywords_str}")

print("\n" + "="*40)
print("Processing complete!")
print(f"Total files with keywords: {len(keyword_results)}")

Found 6 HTML files to process in 'html_outputs'...

Processing: After the Honeymoon.html
  -> Found 5 keywords.

Processing: Attractive Alternative Partners.html
  -> Found 5 keywords.

Processing: Lucky Guy in Love.html
  -> Found 6 keywords.

Processing: output_1.html
  -> Found 6 keywords.

Processing: output_2.html
  -> Found 5 keywords.

Processing: output_3.html
  -> Found 5 keywords.

Source File               | Keywords
--------------------------|-----------
After the Honeymoon.html  | dopamine, fMRI, marriage, pair-bonds, romantic love
Attractive Alternative Partners.html | attention to alternatives, close relationship, romantic love, self-expansion, social neuroscience
Lucky Guy in Love.html    | AI, MPFC, aMCC, fMRI, intrasexual competition, pain empathy
output_1.html             | AI, MPFC, aMCC, fMRI, intrasexual competition, pain empathy
output_2.html             | dopamine, fMRI, marriage, pair-bonds, romantic love
output_3.html             | attention to alternatives, c

# LM Studio API

In [11]:
# See all the available models:
import json

url = "http://127.0.0.1:1234/v1/models"
response = requests.get(url)
models = response.json()
print(json.dumps(models, indent=4))

{
    "data": [
        {
            "id": "google/gemma-3-4b",
            "object": "model",
            "owned_by": "organization_owner"
        },
        {
            "id": "text-embedding-nomic-embed-text-v1.5",
            "object": "model",
            "owned_by": "organization_owner"
        }
    ],
    "object": "list"
}


In [14]:
import requests
import json

url = "http://127.0.0.1:1234/v1/chat/completions"

payload = {
    "model": "gemma-3-4b-it-qat",   # must match one of your loaded models
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "1+1=?"}
    ]
}

response = requests.post(url, json=payload)
data = response.json()

print(json.dumps(data, indent=4))  # print whole response to inspect
print(data["choices"][0]["message"]["content"])  # the assistant's reply


{
    "id": "chatcmpl-ghyy09tkqoto8s43cnapfi",
    "object": "chat.completion",
    "created": 1758100954,
    "model": "google/gemma-3-4b",
    "choices": [
        {
            "index": 0,
            "message": {
                "role": "assistant",
                "content": "1 + 1 = 2\n",
                "tool_calls": []
            },
            "logprobs": null,
            "finish_reason": "stop"
        }
    ],
    "usage": {
        "prompt_tokens": 20,
        "completion_tokens": 9,
        "total_tokens": 29
    },
    "stats": {},
    "system_fingerprint": "google/gemma-3-4b"
}
1 + 1 = 2



In [1]:
import requests
import json
import base64

url = "http://127.0.0.1:1234/v1/chat/completions"

# Read image and encode to base64
with open("After the Honeymoon Table 3.jpg", "rb") as f:
    image_base64 = base64.b64encode(f.read()).decode("utf-8")

payload = {
    "model": "gemma-3-4b-it-qat",   # <-- make sure this is a vision-capable model
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What's in this image?"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{image_base64}"
                    }
                }
            ]
        }
    ]
}

response = requests.post(url, json=payload)
data = response.json()

print(json.dumps(data, indent=4))  # inspect full response
print(data["choices"][0]["message"]["content"])


{
    "id": "chatcmpl-rnjps7u5e8otzryqkdufzl",
    "object": "chat.completion",
    "created": 1758101396,
    "model": "google/gemma-3-4b",
    "choices": [
        {
            "index": 0,
            "message": {
                "role": "assistant",
                "content": "This image is a table from a research paper, likely related to neuroscience or psychology. Here\u2019s a breakdown of what it shows:\n\n*   **Title:** \"Brain responses replicated at Times 1 and 2\" - This suggests the study looked at brain activity changes over two time points (likely before and after an intervention).\n*   **Columns:** The table presents data for different brain regions, along with statistical measures. Let's break down the columns:\n\n    *   **Brain Region:** Lists specific areas of the brain being studied (e.g., SN lateral, Paraentral lobule, Inferior frontal gyrus).\n    *   **x, y:** These likely represent coordinates within the brain region \u2013 potentially used for spatial analysis

In [1]:
import requests
import json
import base64

url = "http://127.0.0.1:1234/v1/chat/completions"

# Read image and encode to base64
with open("After the Honeymoon Table 3.jpg", "rb") as f:
    image_base64 = base64.b64encode(f.read()).decode("utf-8")

payload = {
    "model": "gemma-3-4b-it-qat",   # <-- make sure this is a vision-capable model
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "This table contains XYZ coordinates of brain regions from a neuroscience study. Please extract the 'Region', 'X', 'Y', 'Z' columns only."},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{image_base64}"
                    }
                }
            ]
        }
    ]
}

response = requests.post(url, json=payload)
data = response.json()

print(json.dumps(data, indent=4))  # inspect full response
print(data["choices"][0]["message"]["content"])


{
    "id": "chatcmpl-66aobvpvtsag1yl8c5vpe6",
    "object": "chat.completion",
    "created": 1758104520,
    "model": "google/gemma-3-4b",
    "choices": [
        {
            "index": 0,
            "message": {
                "role": "assistant",
                "content": "Here's the extracted data from the table:\n\n| Region                | X  | Y  | Z  |\n|-----------------------|----|----|----|\n| SN, lateral          | 15 | -15 | -12 |\n| Parainterial lobule   | -6 | -24 | 57 |\n| Inferior frontal gyrus | 54 | 21 | 3  |",
                "tool_calls": []
            },
            "logprobs": null,
            "finish_reason": "stop"
        }
    ],
    "usage": {
        "prompt_tokens": 48,
        "completion_tokens": 96,
        "total_tokens": 144
    },
    "stats": {},
    "system_fingerprint": "google/gemma-3-4b"
}
Here's the extracted data from the table:

| Region                | X  | Y  | Z  |
|-----------------------|----|----|----|
| SN, lateral          | 1

In [ ]:
import requests
import json
import base64
import csv
import re
from pathlib import Path

## ------------------- CONFIGURATION ------------------- ##
# All settings are grouped here for easy access.

# The local API endpoint
API_URL = "http://127.0.0.1:1234/v1/chat/completions"

# The vision-capable model you are using in LM Studio
MODEL_NAME = "gemma-3-vision"

# List of image files to process
IMAGE_FILES = [
    "After the Honeymoon Table 3.jpg",
    "After the Honeymoon Table 4.jpg",
    "After the Honeymoon Table 5.jpg",
    "Attractive Alternative Partners Table 1.jpg",
    "Attractive Alternative Partners Table 3.jpg",
    "Lucky Guy in Love Table 2.jpg",
    "Lucky Guy in Love Table 3.jpg"
]

# The output CSV filename
OUTPUT_CSV = "extracted_xyz_coordinates.csv"

## ------------------- CORE FUNCTIONS ------------------- ##

def process_image(file_path: Path, table_no: str) -> list | None:
    """Encodes an image, sends it to the API, and parses the response."""
    print(f"Processing '{file_path.name}' for Table {table_no}...")
    
    try:
        # 1. Encode image to base64
        with file_path.open("rb") as f:
            image_base64 = base64.b64encode(f.read()).decode("utf-8")
    except Exception as e:
        print(f"  ❌ Error reading or encoding file: {e}")
        return None

    # 2. Prepare prompts and payload for the API
    system_prompt = "You are an expert assistant specializing in extracting structured data from tables in images. Your output must be only a valid JSON array of objects, with no other text."
    user_prompt = "Extract all X, Y, and Z coordinates from this table. Format the output as a JSON array where each object has 'x', 'y', and 'z' keys. For example: `[{\"x\": 10, \"y\": -20, \"z\": 30}]`"
    
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": user_prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_base64}"}}
                ]
            }
        ]
    }

    # 3. Send request and handle potential errors
    try:
        response = requests.post(API_URL, json=payload, timeout=300)
        response.raise_for_status()  # Raises an exception for bad status codes (4xx or 5xx)
        data = response.json()
    except requests.exceptions.RequestException as e:
        print(f"  ❌ API request failed: {e}")
        return None
    except json.JSONDecodeError:
        print(f"  ❌ Failed to decode JSON response from API.")
        return None

    # 4. Safely extract model output and parse it
    try:
        model_output = data["choices"][0]["message"]["content"]
        # The LLM sometimes wraps the JSON in markdown backticks
        cleaned_output = re.sub(r'```json\n?|```', '', model_output).strip()
        coords = json.loads(cleaned_output)
        
        # Add the table number to each coordinate dictionary
        for row in coords:
            row["table_no"] = table_no
        
        print(f"  ✅ Successfully extracted {len(coords)} coordinate sets.")
        return coords
    except (KeyError, IndexError):
        print("  ❌ API response was missing expected 'choices' structure.")
        print(f"  -> Raw response: {data}")
        return None
    except json.JSONDecodeError:
        print("  ❌ Could not parse the model's output as JSON.")
        print(f"  -> Model output: {model_output}")
        return None


def save_to_csv(data: list, output_file: str):
    """Saves a list of dictionaries to a CSV file."""
    if not data:
        print("No data was extracted, CSV file will not be created.")
        return

    fieldnames = ["table_no", "x", "y", "z"]
    with open(output_file, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)
    
    print(f"\nExtraction complete. Data saved to '{output_file}'.")


## ------------------- MAIN EXECUTION ------------------- ##

def main():
    """Main script to orchestrate the extraction and saving process."""
    all_extracted_rows = []
    
    for file_name in IMAGE_FILES:
        image_path = Path(file_name)
        
        # Skip files that don't exist
        if not image_path.exists():
            print(f"⚠️  Warning: File not found, skipping: {file_name}")
            continue

        # Extract table number from filename once
        match = re.search(r"Table\s*(\d+)", file_name, re.IGNORECASE)
        table_no = match.group(1) if match else "unknown"
        
        # Process the image and collect the results
        coordinates = process_image(image_path, table_no)
        if coordinates:
            all_extracted_rows.extend(coordinates)
            
    # Save all collected data to a single CSV file
    save_to_csv(all_extracted_rows, OUTPUT_CSV)


if __name__ == "__main__":
    main()

Processing 'After the Honeymoon Table 3.jpg' for Table 3...
  ❌ API request failed: HTTPConnectionPool(host='127.0.0.1', port=1234): Read timed out. (read timeout=300)
Processing 'After the Honeymoon Table 4.jpg' for Table 4...


# Final Result

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup, NavigableString
from pathlib import Path
import re

## ----------------------------------------------------------------
## STEP 1: EXTRACT METADATA FROM ALL HTML FILES
## ----------------------------------------------------------------

INPUT_DIR = Path("html_outputs")
html_files = list(INPUT_DIR.glob("*.html"))
extracted_metadata = []

print("--- Starting Metadata Extraction ---")
for html_file in html_files:
    print(f"Processing: {html_file.name}")
    html_text = html_file.read_text(encoding='utf-8', errors='ignore')
    soup = BeautifulSoup(html_text, 'html.parser')
    
    # Initialize variables for this file
    title = None
    pmid = None
    pmcid = None
    keywords = []
    
    # --- Extract Title ---
    title_tag = soup.find('h1', class_='heading-title')
    if title_tag:
        title = title_tag.get_text(strip=True)

    # --- Extract PMID and PMCID from the keywords meta tag ---
    keywords_meta_tag = soup.find('meta', attrs={'name': 'keywords'})
    if keywords_meta_tag:
        content_str = keywords_meta_tag.get('content', '')
        for part in content_str.split(','):
            cleaned_part = part.strip()
            if cleaned_part.startswith('pmid:'):
                pmid = cleaned_part.replace('pmid:', '').strip()
            elif cleaned_part.startswith('PMC'):
                pmcid = cleaned_part
    
    # --- Extract Keywords from the body ---
    keyword_strong_tag = soup.find('strong', string=re.compile(r'Keywords:', re.IGNORECASE))
    if keyword_strong_tag:
        next_element = keyword_strong_tag.next_sibling
        if next_element and isinstance(next_element, NavigableString):
            keyword_str = next_element.strip().rstrip('.')
            keywords = [kw.strip() for kw in keyword_str.split(';')]

    # Store all found data for this file
    extracted_metadata.append({
        'Title': title,
        'PMID': pmid,
        'PMCID': pmcid,
        'Keywords': '; '.join(keywords) # Join list into a single string
    })

# Convert the extracted data into a DataFrame
metadata_df = pd.DataFrame(extracted_metadata)
print("\n--- Metadata Extraction Complete ---")
print("Found the following articles:")
print(metadata_df[['Title', 'PMID']])


## ----------------------------------------------------------------
## STEP 2: LOAD YOUR COORDINATE DATA
## ----------------------------------------------------------------
coords_df = pd.read_csv("coordinates.csv")


## ----------------------------------------------------------------
## STEP 3: MAP THE TITLES TO MERGE THE DATASETS
## ----------------------------------------------------------------

title_map = {
    'After the Honeymoon': 'After the Honeymoon: Neural and Genetic Correlates of Romantic Love in Newlywed Marriages',
    'Attractive Alternative Partners': 'Manipulation of Self-Expansion Alters Responses to Attractive Alternative Partners',
    'Lucky Guy in Love': "Decreased Empathic Responses to the 'Lucky Guy' in Love: The Effect of Intrasexual Competition"
}

# Use the map to create a new 'Title' column in the coords_df for merging
coords_df['Title'] = coords_df['Article'].map(title_map)


## ----------------------------------------------------------------
## STEP 4: MERGE DATAFRAMES AND FINALIZE THE TABLE
## ----------------------------------------------------------------
# Merge the two dataframes using the 'Title' column as the key
final_df = pd.merge(coords_df, metadata_df, on='Title')

# Select and reorder columns to match your desired output
final_df = final_df[[
    'PMID',
    'PMCID',
    'Keywords',
    'Table',
    'X',
    'Y',
    'Z'
]]

print("\n--- Final Merged Table ---")
print(final_df.to_string())

# Optional: Save the final table to a new CSV file
final_df.to_csv('final_merged_data.csv', index=False)
print("\n✅ Successfully saved the final table to 'final_merged_data.csv'")